In [60]:
#!pip install --upgrade gensim
import gensim
import json

from flask import Flask, request

import config
from gensim.summarization.bm25 import get_bm25_weights, BM25
import re

In [90]:
doc1 = "предлага больш ассортимент растен собствен производств а такж новинк эт сезон от известн производител европ доставк в люб точк рб бол наименован опт и сайт лиа клематис калистег жимол японск саженц роз миниатюрн плетист штамбов саженц плодов культур яблон груш слив черешн вишн абрикос персик и луковиц цвет лил гиацинт крокус тюльпа нарцисс амариллис рябчик бол наименован хвойн и декоративн растен ту гортенз можжевельник ел пихт вереск и мног друг комнатн растен лимон мандарин инжир лавр благородн мирт саженц ягодн культур смородин крыжовник голубик жимол малин киз и"
doc2 = "хвойн и листвен растен в брест из питомник гарант предлага больш ассортимент растен бол наименован собствен производств а такж новинк эт сезон от известн производител европ доставк в люб точк рб хвойн и декоративн растен гортенз ту можжевельник ел пихт вереск и мног друг лиа клематис калистег жимол японск саженц плодов культур черешн вишн яблон груш слив абрикос персик и саженц роз миниатюрн плетист штамбов луковиц цвет гиацинт крокус лил тюльпа нарцисс амариллис рябчик бол наименован саженц ягодн культур смородин крыжовник голубик жимол малин киз и комнатн растен лимон лавр благородн мандарин инжир мирт опт и розниц звон сайт"
doc3 = "учител физическ культур и здоров ставк высш педагогическ образован"
doc4 = "на предприят в росс треб работник грузчик разнорабоч работник в кофейн официант уборщиц кассир мойщиц посуд на рыбзавод проживан бесплатн униформ выда вахт ил месяц зп за смен"
query_doc = "учител"


In [91]:
doc1 = re.split(" ", doc1)
doc2 = re.split(" ", doc2)
doc3 = re.split(" ", doc3)
doc4 = re.split(" ", doc4)
query_doc = re.split(" ", query_doc)
corp = [doc1, doc2, doc3, doc4, query_doc]

In [92]:
bm25 = BM25(corp)

In [93]:
average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys())

In [94]:
scores = bm25.get_scores(query_doc, average_idf)

In [96]:
scores[:-1]

[0, 0, 0.5257378697206452, 0]

In [97]:
best_result = corp[scores.index(max(scores[:-1]))]
print(best_result)

['учител', 'физическ', 'культур', 'и', 'здоров', 'ставк', 'высш', 'педагогическ', 'образован']


In [152]:
[x for _,x in sorted(zip(scores[:-1],corp[:-1]), reverse=True)]

[['учител',
  'физическ',
  'культур',
  'и',
  'здоров',
  'ставк',
  'высш',
  'педагогическ',
  'образован'],
 ['хвойн',
  'и',
  'листвен',
  'растен',
  'в',
  'брест',
  'из',
  'питомник',
  'гарант',
  'предлага',
  'больш',
  'ассортимент',
  'растен',
  'бол',
  'наименован',
  'собствен',
  'производств',
  'а',
  'такж',
  'новинк',
  'эт',
  'сезон',
  'от',
  'известн',
  'производител',
  'европ',
  'доставк',
  'в',
  'люб',
  'точк',
  'рб',
  'хвойн',
  'и',
  'декоративн',
  'растен',
  'гортенз',
  'ту',
  'можжевельник',
  'ел',
  'пихт',
  'вереск',
  'и',
  'мног',
  'друг',
  'лиа',
  'клематис',
  'калистег',
  'жимол',
  'японск',
  'саженц',
  'плодов',
  'культур',
  'черешн',
  'вишн',
  'яблон',
  'груш',
  'слив',
  'абрикос',
  'персик',
  'и',
  'саженц',
  'роз',
  'миниатюрн',
  'плетист',
  'штамбов',
  'луковиц',
  'цвет',
  'гиацинт',
  'крокус',
  'лил',
  'тюльпа',
  'нарцисс',
  'амариллис',
  'рябчик',
  'бол',
  'наименован',
  'саженц',
  'яг

In [159]:
app = Flask(__name__)

@app.route('/ranking', methods=["POST"])
def ranging():
    documents = request.json
    corpus = []
    for doc in documents["documents"]:
        text = re.split(" ", doc["text_normalized"])
        corpus.append(text)
    query = re.split(" ", documents["terms"][0]["term"])
    corpus.append(query)
    
    bm25 = BM25(corpus)
    average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys())
    scores = bm25.get_scores(query_doc, average_idf)
    
    sorted_corp = [x for _,x in sorted(zip(scores[:-1],corp[:-1]), reverse=True)]
    documents["documents"] = sorted_corp
    #print(len(documents["documents"]))
    #print(len(scores[:-1]))
    return str(documents)


In [ ]:
if __name__ == "__main__":
    #app.run(port=config.RANKING_PORT)
    app.run(host='0.0.0.0', port=config.RANKING_PORT)



 * Running on http://0.0.0.0:13502/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Nov/2018 19:37:53] "POST /ranking HTTP/1.1" 200 -
